In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
#Read states.csv and convert to dataframe
states_df = pd.read_csv("Resources/states.csv")

#Delete unnecessary columns, reorder columns, and clean column names
states_df = states_df[["STATE", "YEAR", "TOTAL_REVENUE", "TOTAL_EXPENDITURE", "ENROLL"]]
states_df.rename(columns = {"STATE": "State", 
                            "YEAR": "Year", 
                            "TOTAL_REVENUE": "Total Revenues", 
                            "TOTAL_EXPENDITURE": "Total Expenditures", 
                            "ENROLL": "Enrollment"}, inplace = True) 

#Delete rows with missing values, reset index, and display dataframe
states_df = states_df.dropna().reset_index()
del states_df["index"]

#Display the dataframe
states_df


,State,Year,Total Revenues,Total Expenditures,Enrollment
0,Alabama,1993,2827391,2833433,727716.0
1,Alaska,1993,1191398,1126398,121156.0
2,Arizona,1993,3427976,3623946,676297.0
3,Arkansas,1993,1346909,1376067,311432.0
4,California,1993,28043338,28110986,5129788.0
...,...,...,...,...,...
1219,Virginia,2016,16259274,16497520,1283493.0
1220,Washington,2016,14964364,15253296,1083973.0
1221,West Virginia,2016,3391579,3366566,276764.0
1222,Wisconsin,2016,11697466,11787535,857736.0


In [3]:
#Read naep.csv and convert to dataframe
tests_df = pd.read_csv("Resources/naep.csv")

#Delete unnecessary columns, reorder columns, and clean column names
tests_df = tests_df[["STATE", "YEAR", "TEST_YEAR", "TEST_SUBJECT", "AVG_SCORE"]]
tests_df.rename(columns = {"STATE": "State", 
                           "YEAR": "Year",
                           "TEST_YEAR": "Grade Level",
                           "TEST_SUBJECT": "Test Subject", 
                           "AVG_SCORE": "Average Test Score"}, inplace = True)

#Remove rows that have non-null objects and convert to numeric
tests_df = tests_df[pd.to_numeric(tests_df["Average Test Score"], errors="coerce").notnull()]
tests_df[["Average Test Score"]] = tests_df[["Average Test Score"]].apply(pd.to_numeric)

#Split into dataframes based on test subject and then grade level
math_tests_df = tests_df.loc[tests_df["Test Subject"] == "Mathematics", :]
math_tests_fourth_df = math_tests_df.loc[math_tests_df["Grade Level"] == 4,:]
math_tests_eighth_df = math_tests_df.loc[math_tests_df["Grade Level"] == 8,:]
reading_tests_df = tests_df.loc[tests_df["Test Subject"] == "Reading", :]
reading_tests_fourth_df = reading_tests_df.loc[reading_tests_df["Grade Level"] == 4,:]
reading_tests_eighth_df = reading_tests_df.loc[reading_tests_df["Grade Level"] == 8,:]

#Delete unnecessary columns and rename columns
math_tests_fourth_df = math_tests_fourth_df[["State", "Year", "Average Test Score"]].rename(columns = {"Average Test Score": "Average Math Score (4th Grade)"})
math_tests_eighth_df = math_tests_eighth_df[["State", "Year", "Average Test Score"]].rename(columns = {"Average Test Score": "Average Math Score (8th Grade)"})
reading_tests_fourth_df = reading_tests_fourth_df[["State", "Year", "Average Test Score"]].rename(columns = {"Average Test Score": "Average Reading Score (4th Grade)"})
reading_tests_eighth_df = reading_tests_eighth_df[["State", "Year", "Average Test Score"]].rename(columns = {"Average Test Score": "Average Reading Score (8th Grade)"})

#Merge into single dataframe using state-by-state financial data
merged_math_df = pd.merge(math_tests_fourth_df, math_tests_eighth_df, on = ["State", "Year"], how = "outer")
merged_reading_df = pd.merge(reading_tests_fourth_df, reading_tests_eighth_df, on = ["State", "Year"], how = "outer")
merged_tests_df = pd.merge(merged_math_df, merged_reading_df, on = ["State", "Year"], how = "outer")
merged_state_data = pd.merge(states_df, merged_tests_df, on = ["State", "Year"], how = "outer")
merged_state_data = merged_state_data.sort_values(["State", "Year"]).set_index(["State", "Year"])

#Display the dataframe
merged_state_data

Total Revenues  Total Expenditures  Enrollment  \
State   Year                                                   
Alabama 1990             NaN                 NaN         NaN   
        1992             NaN                 NaN         NaN   
        1993       2827391.0           2833433.0    727716.0   
        1994       2970223.0           3103266.0    730004.0   
        1995       3367437.0           3353363.0    727480.0   
...                      ...                 ...         ...   
Wyoming 2013       1695967.0           1675477.0     91533.0   
        2014       1772633.0           1775999.0     92732.0   
        2015       1962874.0           1942406.0     93867.0   
        2016       2044669.0           2034229.0     94511.0   
        2017             NaN                 NaN         NaN   

              Average Math Score (4th Grade)  Average Math Score (8th Grade)  \
State   Year                                                                   
Alabama 1990                             NaN                      252.863660   
        1992                      208.327876                      252.187522   
        1993                             NaN                             NaN   
        1994                             NaN                             NaN   
        1995                             NaN                             NaN   
...                                      ...                             ...   
Wyoming 2013                      246.522401                      288.119137   
        2014                             NaN                             NaN   
        2015                      246.759764                      286.666547   
        2016                             NaN                             NaN   
        2017                      247.820619                      288.761812   

              Average Reading Score (4th Grade)  \
State   Year                                      
Alabama 1990                                NaN   
        1992                         207.358492   
        1993                                NaN   
        1994                         207.963517   
        1995                                NaN   
...                                         ...   
Wyoming 2013                         225.848975   
        2014                                NaN   
        2015                         228.160803   
        2016                                NaN   
        2017                         226.831211   

              Average Reading Score (8th Grade)  
State   Year                                     
Alabama 1990                                NaN  
        1992                                NaN  
        1993                                NaN  
        1994                                NaN  
        1995                                NaN  
...                                         ...  
Wyoming 2013                         270.966172  
        2014                                NaN  
        2015                         268.803363  
        2016                                NaN  
        2017                         269.020028  

[1365 rows x 7 columns]

In [4]:
merged_state_data.to_csv("Resources/merged_state_data.csv", index = True, header = True)
